# Проект "Определение уровня английского для фильмов"

В данном проекте необходимо определить, какой уровень английского языка представлен в фильме. Уровень английского языка определяется в соответствии с уровнем Oxford CEFR. Определение уровня осуществляется на основе субтитров к фильму. В дополнение, можно добавлять фильмы и субтитры к ним из открытых источников, поскольку первоначально в дасете 241 фильм.

**План работы:**
1. [Предобработка данных](#data_preprocessing)
2. [Расширение датасета](#dataset_expansion)
3. [Выбор метрики](#metric)
4. [Создание модели](#model)
5. [Анализ результатов](#analysis)

<a name='data_preprocessing'></a>
## 1. Предобработка данных

### 1. 1. Общий файл с фильмами

In [2]:
# Импортирование необходимых библиотек
import numpy as np
import pandas as pd
import PyPDF2
import pysrt
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/midle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/midle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Импорт файла
df_movies = pd.read_excel(
    '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/English_scores/movies_labels.xlsx',
)

df_movies.drop('id', axis=1, inplace=True)
df_movies.columns = ['movie', 'level']

df_movies.head()

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [4]:
# Заменим промежуточный уровень на нижний
df_movies.loc[df_movies['level'] == 'A2/A2+', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'A2/A2+, B1', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'B1, B2', 'level'] = 'B1'

df_movies['level_num'] = df_movies['level'].map({
    'A2': 0,
    'B1': 1,
    'B2': 2,
    'C1': 3
})

df_movies.head(5)

,movie,level,level_num
0,10_Cloverfield_lane(2016),B1,1
1,10_things_I_hate_about_you(1999),B1,1
2,A_knights_tale(2001),B2,2
3,A_star_is_born(2018),B2,2
4,Aladdin(1992),A2,0


In [5]:
# Пропущенные значения
df_movies.isna().sum()

movie        0
level        0
level_num    0
dtype: int64

In [6]:
# Распределение количества фильмов по уровням 
df_movies['level'].value_counts().sort_values(ascending=False)

B2    101
B1     63
C1     40
A2     37
Name: level, dtype: int64

### 1. 2. Файлы с субтитрами

In [ ]:
# Функция для загрузки субтитров и их лемматизация
def saving_subs(film_loc):
    # Загрузка субтитров по кодированию
    film_subs = []
    encodings = ['', 'UTF-8-SIG', 'ISO-8859-1', 'utf-8', 'Windows-1252', 'ascii']
    encoding_number = 0

    # Проверка возможности правильности кодирования, иначе - использование другого
    while not film_subs:
        try:
            film_subs = pysrt.open(
                film_loc,
                encoding=encodings[encoding_number]
            )
        except UnicodeDecodeError:
            encoding_number += 1
    
    HTML = r'<.*?>' # html тэги меняем на пробел
    TAG = r'{.*?}' # тэги меняем на пробел
    COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках меняем на пробел
    UPPER = r'[[A-Za-z ]+[\:\]]' # указания на того кто говорит (BOBBY:)
    LETTERS = r'[^a-zA-Z\'.,!? ]' # все что не буквы меняем на пробел 
    SPACES = r'([ ])\1+' # повторяющиеся пробелы меняем на один пробел
    DOTS = r'[\.]+' # многоточие меняем на точку
    SYMB = r"[^\w\d'\s]" # знаки препинания кроме апострофа

    def clean_subs(subs):
        subs = subs[1:] # удаляем первый рекламный субтитр
        txt = re.sub(HTML, ' ', subs.text) # html тэги меняем на пробел
        txt = re.sub(COMMENTS, ' ', txt) # комменты в скобках меняем на пробел
        txt = re.sub(UPPER, ' ', txt) # указания на того кто говорит (BOBBY:)
        txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
        txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
        txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
        txt = re.sub(SYMB, '', txt) # знаки препинания кроме апострофа на пустую строку
        txt = re.sub('www', '', txt) # кое-где остаётся www, то же меняем на пустую строку
        txt = txt.lstrip() # обрезка пробелов слева
        txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы 
        txt = txt.lower() # текст в нижний регистр
        return txt

    # Объединение текста воедино, убирая лишнее
    for sub in film_subs:
        sub_text = str(sub.text).split('\n')
        try:
            for i in range(len(sub_text)):
                if not sub_text[i][0] == '(' and not sub_text[i][-1] == ')' and not sub_text[i][0] == '<' \
                and not sub_text[i][-1] == '>' and not sub_text[i][0] == '[' and not sub_text[i][-1] == ']':
                    film_text += sub_text[i]
                    film_text += ' '
        except IndexError: 
            pass

    # Токенизация по словам
    film_words_tokens = nltk.word_tokenize(film_text)

    numbers_and_symbols = ['.', ',', '!', ':', ';', '?', '_', '__', '...', '#', '--', '``', 'âª', '\n', 'br/', \
                           '<', '>', '"', '=', '/', '\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    film_words = []

    # Лемматизация слов и добавление их в отдельный список
    lemmatizer = WordNetLemmatizer()
    for i in range(len(film_words_tokens)):
        word = lemmatizer.lemmatize(film_words_tokens[i])
        # Проверка на наличие слова в списке
        if word not in film_words:
            # Использование только слов без цифр и символов
            if word.startswith("'"):
                film_words.append('to be')
            elif word == "n't":
                film_words.append('not')
            elif word == "y'all":
                film_words.append('you')
                film_words.append('all')

            if not re.search('\W', word) and word not in numbers_and_symbols:
                film_words.append(word)
    
    # Объединение слов в одну строчку
    film_words = " ".join(film_words)
    return film_words

# Функция для загрузки субтитров и их лемматизация
def saving_subs(film_loc):
    # Загрузка субтитров по кодированию
    film_subs = []
    encodings = ['', 'UTF-8-SIG', 'ISO-8859-1', 'utf-8', 'Windows-1252', 'ascii']
    encoding_number = 0

    # Проверка возможности правильности кодирования, иначе - использование другого
    while not film_subs:
        try:
            film_subs = pysrt.open(
                film_loc,
                encoding=encodings[encoding_number]
            )
        except UnicodeDecodeError:
            encoding_number += 1

    film_text = ''

    # Объединение текста воедино, убирая лишнее
    for sub in film_subs:
        sub_text = str(sub.text).split('\n')
        try:
            for i in range(len(sub_text)):
                if not sub_text[i][0] == '(' and not sub_text[i][-1] == ')' and not sub_text[i][0] == '<' \
                and not sub_text[i][-1] == '>' and not sub_text[i][0] == '[' and not sub_text[i][-1] == ']':
                    film_text += sub_text[i]
                    film_text += ' '
        except IndexError: 
            pass

    # Токенизация по словам
    film_words_tokens = nltk.word_tokenize(film_text)

    numbers_and_symbols = ['.', ',', '!', ':', ';', '?', '_', '__', '...', '#', '--', '``', 'âª', '\n', 'br/', \
                           '<', '>', '"', '=', '/', '\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    film_words = []

    # Лемматизация слов и добавление их в отдельный список
    lemmatizer = WordNetLemmatizer()
    for i in range(len(film_words_tokens)):
        word = lemmatizer.lemmatize(film_words_tokens[i])
        # Проверка на наличие слова в списке
        if word not in film_words:
            # Использование только слов без цифр и символов
            if word.startswith("'"):
                film_words.append('to be')
            elif word == "n't":
                film_words.append('not')
            elif word == "y'all":
                film_words.append('you')
                film_words.append('all')

            if not re.search('\W', word) and word not in numbers_and_symbols:
                film_words.append(word)
    
    # Объединение слов в одну строчку
    film_words = " ".join(film_words)
    return film_words

In [10]:
# Получение субтитров для каждого фильма, воспользовавшись функцией для обработки субтитров
for index, row in df_movies.iterrows():
    folders = ['A2', 'B1', 'B2', 'C1', 'Subtitles']
    folders_num = 0
    dir_found = False
    
    while not dir_found:
        try:
            subs = saving_subs(f"/Users/midle/Desktop/Coding/Data Science/Projects/English_level/English_scores/Subtitles_all/{folders[folders_num]}/{row['movie']}.srt")
        except FileNotFoundError:
            folders_num += 1
        except IndexError:
            dir_found = True
        else:
            df_movies.loc[df_movies['movie'] == row['movie'], 'subs'] = subs
            dir_found = True

<class 'pysrt.srtfile.SubRipFile'>
<class 'pysrt.srtfile.SubRipFile'>
<class 'pysrt.srtfile.SubRipFile'>
<class 'pysrt.srtfile.SubRipFile'>
<class 'pysrt.srtfile.SubRipFile'>
<class 'pysrt.srtfile.SubRipFile'>


KeyboardInterrupt: 

In [ ]:
# Удаление фильмов, к которым не были найдены субтитры
df_movies = df_movies[(df_movies['subs'] != '') & (df_movies['subs'].notna())]

### 1. 3. Слова по уровню языка

Создадим список со всеми словами, а также отдельный словарь с ключами в виде уровней английского и значениями - индексами начала слов уровня - 1.

In [ ]:
# Ссылки на слова (А1-C1)
a1_b2_link = '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/Oxford_CEFR_level/The_Oxford_3000_by_CEFR_level.pdf'
b2_c1_link = '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/Oxford_CEFR_level/The_Oxford_5000_by_CEFR_level.pdf'

In [ ]:
# Функция для сохранения слов в список из PDF-файла 
def get_level_words(link):
    reader = PyPDF2.PdfReader(link)
    
    words = ''
    for n in range(len(reader.pages)):
        words += reader.pages[n].extract_text()
        
    words = words.split('\n')
    words = [words[n].split()[0] for n in range(len(words)) if n > 1]
    return words

In [ ]:
# Сохраним слова в отдельные переменные
a1_b2_words = get_level_words(a1_b2_link)
b2_c1_words = get_level_words(b2_c1_link)

# Внесем некоторые корректировки
a1_b2_words[1] = 'a'
a1_b2_words.insert(2, 'an')
b2_c1_words.remove('3000')
b2_c1_words.remove('B2')

In [ ]:
# Создадим единый список слов

level_words = a1_b2_words
level_words.extend(b2_c1_words)
level_words[:10]

In [ ]:
# Создадим словарь с ключами в виде уровня английского и значениями в виде индексов начала слов
# данного уровня минус единица. Получается, что конец слов по уровню - значение следующего уровня минус единица.
levels = ['A1', 'A2', 'B1', 'B2', 'C1']
levels_dict = {level: level_words.index(level) for level in levels}

print(levels_dict)

In [ ]:
print(level_words[3958:])

In [ ]:
# Определение количества слов по уровням для каждого фильма
for index, row in df_movies.iterrows():
    # Словарь с подсчетом слов по каждому уровню для отдельного фильма
    film_level_words = {level: [] for level in levels}
    
    # Добавление в словарь слов по их уровням
    for word in row['subs'].split():
        try:
            word_index = level_words.index(word.lower())
            if levels_dict['A1'] < word_index < levels_dict['A2']:
                if word not in film_level_words['A1']:
                    film_level_words['A1'].append(word)
            elif levels_dict['A2'] < word_index < levels_dict['B1']:
                if word not in film_level_words['A2']:
                    film_level_words['A2'].append(word)
            elif levels_dict['B1'] < word_index < levels_dict['B2']:
                if word not in film_level_words['B1']:
                    film_level_words['B1'].append(word)
            elif levels_dict['B2'] < word_index < levels_dict['C1']:
                if word not in film_level_words['B2']:
                    film_level_words['B2'].append(word)
            else:
                if word not in film_level_words['C1']:
                    film_level_words['C1'].append(word)
        except ValueError:
            pass
    
    # Подсчет количества слов по уровням
    levels_words_count = {level: len(film_level_words[level]) for level in levels}
    
    # Добавления количества слов по уровням в отдельные столбцы
    for level in levels:
        df_movies.loc[df_movies['movie'] == row['movie'], level] = levels_words_count[level]

In [ ]:
# Получения значения количества слов по уровням для отдельного фильмы
'''
movie = df_movies[df_movies['movie'] == 'Suits.S03E06.720p.HDTV.x264-mSD']
total_words = movie[levels].sum(axis=1)
words_by_previous_levels = 0

for level in levels:    
    if ((movie[level].values + words_by_previous_levels) / total_words >= 0.75).values[0]:
        print(level)
        break
    else:
        words_by_previous_levels += movie[level].values
'''

<a name='dataset_expansion'></a>
## 2. Расширение датасета

<a name='metric'></a>
## 3. Выбор метрики

Имеем дело с задачей мультиклассификации. В связи с этим необходимо использовать соответствующие метрики определения качества моделей. Назначение проекта - помочь учащимся, изучающих английский язык. В связи с этим лучше всего использовать F1-меру, поскольку она сочетает в себе и точность (precision), и полноту (recall). Однако, поскольку мы имеем дело с мультиклассификацией, то необходимо воспользоваться "Макро F1-мерой" (деомнстрирующей среднее значение F1-меры по классам) и "Микро F1-мерой" (глобальная средняя F1-меры, вычисляющая сумму TP, FN и FP).

Таким образом, для определения качества моделей будем использовать **f1_micro** и **f1_macro**.

<a name='model'></a>
## 4. Создание модели

### 4. 1. Разделение выборок

In [ ]:
# Разделение датасета на тренировочную и тестовую выборки

# X = df_movies.drop(['movie', 'level', 'level_num'], axis=1)
X = df_movies['subs']
y = df_movies['level_num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_movies['level_num']
)

In [ ]:
# Функция для нахождения предсказаний
def get_y_pred(model):
    vectorizer = TfidfVectorizer(stop_words={'english'})
    vec_col = ['subs']
    scaler = StandardScaler()
    
#     preprocessing = ColumnTransformer([
#         ('vectorizer', vectorizer, vec_col),
#         ('scaler', scaler, levels),
#     ])
    
#     pipe = Pipeline([
#         ('preprocessing', preprocessing),
#         ('model', model)
#     ])
    
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('model', model)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    return y_pred

In [ ]:
def print_f1_score(model, pred):
    # Проверка DecisionTreeClassifier на тестовой выборке
    print(f"{model}:")
    print("\tf1_micro:", f1_score(y_test, pred, average='micro'))
    print("\tf1_macro:", f1_score(y_test, pred, average='macro'))

### 4. 2. DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(random_state=42)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

### 4. 3. RandomForestClassifier

In [ ]:
model = RandomForestClassifier(random_state=42)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

### 4. 4. LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=300)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

<a name='analysis'></a>
## 5. Анализ результатов